<a href="https://colab.research.google.com/github/prawees/marmoset_project/blob/main/MarmosetProject_6402001_06_09_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://zenodo.org/records/5849371/files/marmoset-dlc-2021-05-07.zip -O marmoset_data.zip
#Download dataset and name it marmoset_data.zip
!unzip marmoset_data.zip -d ./marmoset
#Unzip file
!pip install ultralytics deeplabcut2yolo
#Install ultralytics and deeplabcut2yolo

Streaming output truncated to the last 5000 lines.
  inflating: ./marmoset/marmoset-dlc-2021-05-07/labeled-data/reachingvideo1/img03515.png  
  inflating: ./marmoset/__MACOSX/marmoset-dlc-2021-05-07/labeled-data/reachingvideo1/._img03515.png  
  inflating: ./marmoset/marmoset-dlc-2021-05-07/labeled-data/reachingvideo1/img06445.png  
  inflating: ./marmoset/__MACOSX/marmoset-dlc-2021-05-07/labeled-data/reachingvideo1/._img06445.png  
  inflating: ./marmoset/marmoset-dlc-2021-05-07/labeled-data/reachingvideo1/img00034.png  
  inflating: ./marmoset/__MACOSX/marmoset-dlc-2021-05-07/labeled-data/reachingvideo1/._img00034.png  
  inflating: ./marmoset/marmoset-dlc-2021-05-07/labeled-data/reachingvideo1/img04252.png  
  inflating: ./marmoset/__MACOSX/marmoset-dlc-2021-05-07/labeled-data/reachingvideo1/._img04252.png  
  inflating: ./marmoset/marmoset-dlc-2021-05-07/labeled-data/reachingvideo1/img02623.png  
  inflating: ./marmoset/__MACOSX/marmoset-dlc-2021-05-07/labeled-data/reachingvideo1/.

In [ ]:
import deeplabcut2yolo

json = "./marmoset/marmoset-dlc-2021-05-07/training-datasets/iteration-0/UnaugmentedDataSet_marmosetMay7/dlc_shuffle1_train.json"
#path to the DeepLabCut annotation file (.json format)
csv = "./marmoset/marmoset-dlc-2021-05-07/training-datasets/iteration-0/UnaugmentedDataSet_marmosetMay7/CollectedData_dlc.csv"
#path to the .csv file
save = "./marmoset/marmoset-dlc-2021-05-07/labeled-data/"
#the directory where the converted YOLO-compatible data will be saved

deeplabcut2yolo.convert(json, csv, save, datapoint_classes=[0, 1], n_keypoint_per_datapoint=30, precision=6)
#Use fucntion deeplabcut2yolo.convert() to convert deeplabcut data into YOLO-compatible data
#datapoint_classes = classes labels for the datapoints (in this case there are two monkeys to detect)
#n_keypoint_per_datapoint = number of keypoints per datapoint
#precision = decimal precision for the converted annotations

,height,file_name,width,id,normalized_coords,0_coords,data_0,0_bbox,0_x,0_y,0_w,0_h,1_coords,data_1,1_bbox,1_x,1_y,1_w,1_h
0,500,reachingvideo1_img01053.png,500,1690,"[0.44621944721940004, 0.4358964368964, 0.44990...","[0.44621944721940004, 0.4358964368964, 0.44990...",0.446219 0.435896 1 0.449900 0.490226 1 0.5097...,"[0.44621944721940004, 0.4245175839066, 0.70595...",0.576090,0.52095,0.259740,0.192864,"[None, None, None, None, None, None, None, Non...",0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,None,0.000000,0.000000,0.000000,0.000000
1,300,reachingvideo1_img05399.png,400,8487,"[None, None, None, None, None, None, None, Non...","[None, None, None, None, None, None, None, Non...",0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,None,0.000000,0.00000,0.000000,0.000000,"[0.4925, 0.32, 0.4425, 0.39666666666666667, 0....",0.492500 0.320000 1 0.442500 0.396667 1 0.5550...,"[0.4425, 0.32, 0.635, 0.7433333333333333]",0.538750,0.531667,0.192500,0.423333
2,500,reachingvideo1_img01711.png,500,2771,"[None, None, None, None, None, None, None, Non...","[None, None, None, None, None, None, None, Non...",0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,None,0.000000,0.00000,0.000000,0.000000,"[None, None, 0.6443546453546, 0.25274625374619...",0 0 0 0.644355 0.252746 1 0.727605 0.292706 1 ...,"[0.5261395271396, 0.24275624375619997, 0.72760...",0.626872,0.600732,0.201465,0.715951
3,500,reachingvideo1_img01516.png,500,2403,"[None, None, None, None, None, None, None, Non...","[None, None, None, None, None, None, None, Non...",0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,None,0.000000,0.00000,0.000000,0.000000,"[0.5178145188145999, 0.7322667332668, 0.556109...",0.517815 0.732267 1 0.556110 0.660672 1 0.4495...,"[0.3787293701172, 0.3243413253414, 0.562769563...",0.470749,0.528304,0.184040,0.407925
4,600,reachingvideo1_img02382.png,600,4057,"[None, None, None, None, None, None, None, Non...","[None, None, None, None, None, None, None, Non...",0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,None,0.000000,0.00000,0.000000,0.000000,"[0.49666666666666665, 0.525, 0.543333333333333...",0.496667 0.525000 1 0.543333 0.498333 1 0.4616...,"[0.4166666666666667, 0.27166666666666667, 0.55...",0.487500,0.398333,0.141667,0.253333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3959,400,reachingvideo1_img06891.png,400,10630,"[0.51, 0.6, 0.545, 0.5425, 0.465, 0.5625, 0.50...","[0.51, 0.6, 0.545, 0.5425, 0.465, 0.5625, 0.50...",0.510000 0.600000 1 0.545000 0.542500 1 0.4650...,"[0.4425, 0.3925, 0.545, 0.6]",0.493750,0.49625,0.102500,0.207500,"[0.5775, 0.505, 0.5925, 0.56, 0.62, 0.4875, 0....",0.577500 0.505000 1 0.592500 0.560000 1 0.6200...,"[0.5775, 0.4375, 0.6925, 0.56]",0.635000,0.498750,0.115000,0.122500
3960,400,reachingvideo1_img06506.png,400,10245,"[0.73, 0.645, 0.805, 0.585, 0.6725, 0.5875, 0....","[0.73, 0.645, 0.805, 0.585, 0.6725, 0.5875, 0....",0.730000 0.645000 1 0.805000 0.585000 1 0.6725...,"[0.4425, 0.4, 0.805, 0.7125]",0.623750,0.55625,0.362500,0.312500,"[0.295, 0.2875, 0.255, 0.35, 0.3625, 0.3125, 0...",0.295000 0.287500 1 0.255000 0.350000 1 0.3625...,"[0.255, 0.2875, 0.535, 0.6025]",0.395000,0.445000,0.280000,0.315000
3961,500,reachingvideo1_img04631.png,500,7184,"[None, None, None, None, None, None, None, Non...","[None, None, None, None, None, None, None, Non...",0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,None,0.000000,0.00000,0.000000,0.000000,"[0.3606403596404, 0.6573416583416, 0.423080609...",0.360640 0.657342 1 0.423081 0.693122 1 0.3871...,"[0.32900532800540006, 0.4059264069264, 0.68826...",0.508633,0.549524,0.359256,0.287196
3962,500,reachingvideo1_img01332.png,500,2121,"[0.5278045288045999, 0.6590066600066, 0.561104...","[0.5278045288045999, 0.6590066600066, 0.561104...",0.527805 0.659007 1 0.561105 0.602397 1 0.4711...,"[0.44592902565000003, 0.42323251140120005, 0.5...",0.518502,0.54112,0.145146,0.235774,"[None, None, None, None, None, None, None, Non...",0 0 0 0 0 0 0 0 0 0 0 0 0

In [ ]:
import yaml

data = {
    "train": "/content/marmoset/marmoset-dlc-2021-05-07/labeled-data/reachingvideo1",
    #path to the data that will use to train the model
    "val": "/content/marmoset/marmoset-dlc-2021-05-07/labeled-data/refinement1",
    #path to the data that will use to validate the model
    "kpt_shape": [15, 3],
    #keypoint shape: in this case there are 15 keypoints per class  and each keypoints hold 3 values (x, y, confidence score)
    "flip_idx":  [0, 3, 2, 1, 6, 7, 4, 5, 9, 8, 11, 10, 12, 13, 14],
    #flip index: defines which keypoints should be swapped when the image is horizontally flipped (mirrored)
    "nc": 2,
    #number of classes
    "names" : ["B", "W"]
    #name of classes
    }

output= "/content/output.yaml"
#the directory where the yml data will be saved

with open(output, "w") as file:
    yaml.dump(data, file, default_flow_style=None, sort_keys=False)

In [ ]:
from ultralytics import YOLO

modelyolo = YOLO("yolov8n-pose.pt")

modelyolo.train(data="output.yaml", epochs=20)
#train YOLO model


100%|██████████| 6.52M/6.52M [00:00<00:00, 143MB/s]


Ultralytics YOLOv8.2.90 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=pose, mode=train, model=yolov8n-pose.pt, data=output.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, 

100%|██████████| 755k/755k [00:00<00:00, 22.2MB/s]


Overriding model.yaml kpt_shape=[17, 3] with kpt_shape=[15, 3]
Overriding model.yaml nc=1 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True

100%|██████████| 6.25M/6.25M [00:00<00:00, 97.8MB/s]


...

                   all        424        652      0.521       0.58      0.526      0.311      0.108     0.0552     0.0183     0.0021



20 epochs completed in 0.817 hours.
Optimizer stripped from runs/pose/train/weights/last.pt, 6.7MB
Optimizer stripped from runs/pose/train/weights/best.pt, 6.7MB

Validating runs/pose/train/weights/best.pt...
Ultralytics YOLOv8.2.90 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-pose summary (fused): 187 layers, 3,248,633 parameters, 0 gradients, 9.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:14<00:00,  1.01s/it]


                   all        424        652       0.52       0.58      0.526      0.311      0.108     0.0552     0.0182    0.00209
                     B        326        326       0.49      0.595      0.504       0.28      0.105     0.0583     0.0246    0.00289
                     W        326        326       0.55      0.564      0.548      0.342      0.111     0.0521     0.0118    0.00129
Speed: 0.3ms preprocess, 3.1ms inference, 0.0ms loss, 5.1ms postprocess per image
Results saved to runs/pose/train


ultralytics.utils.metrics.PoseMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e4350d758a0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(P)', 'F1-Confidence(P)', 'Precision-Confidence(P)', 'Recall-Confidence(P)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041, 

In [ ]:
!wget https://zenodo.org/records/8437121/files/marmoset-dlc-2021-05-07.zip -O marmoset_test.zip
!unzip marmoset_test.zip -d ./marmoset_test

--2024-09-08 13:05:15--  https://zenodo.org/records/8437121/files/marmoset-dlc-2021-05-07.zip
Resolving zenodo.org (zenodo.org)... 188.184.103.159, 188.184.98.238, 188.185.79.172, ...
Connecting to zenodo.org (zenodo.org)|188.184.103.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 814036526 (776M) [application/octet-stream]
Saving to: ‘marmoset_test.zip’

marmoset_test.zip   100%[===================>] 776.33M  17.5MB/s    in 47s     

2024-09-08 13:06:03 (16.5 MB/s) - ‘marmoset_test.zip’ saved [814036526/814036526]

Archive:  marmoset_test.zip
   creating: ./marmoset_test/marmoset-dlc-2021-05-07/
  inflating: ./marmoset_test/__MACOSX/._marmoset-dlc-2021-05-07  
   creating: ./marmoset_test/marmoset-dlc-2021-05-07/labeled-data/
  inflating: ./marmoset_test/__MACOSX/marmoset-dlc-2021-05-07/._labeled-data  
   creating: ./marmoset_test/marmoset-dlc-2021-05-07/labeled-data/refinement1/
  inflating: ./marmoset_test/__MACOSX/marmoset-dlc-2021-05-07/labeled-data/

In [ ]:
from ultralytics import SAM

# Load a model
modelsam = SAM("sam_b.pt")

In [ ]:
import os

# Define input and output directories
input_dir = "./marmoset_test/marmoset-dlc-2021-05-07/labeled-data/reachingvideo1"
output_dir = "./marmoset_test/marmoset-dlc-2021-05-07/labeled-data/output"

# Create output directory if it doesn't exist
if os.path.exists(output_dir) == False:
    os.makedirs(output_dir)

for filename in os.listdir(input_dir):
    if filename.lower().endswith(('.png')):
        # Path to the image
        image_path = os.path.join(input_dir, filename)

        # Run YOLO model
        model_output = modelyolo(image_path)
        keypoints = model_output[0].keypoints.xy

        # Check bounding boxes
        if len(keypoints) == 0:
            print(f"No {filename}")
            continue

        # Convert bounding boxes to list format suitable for SAM
        keypoints_list = keypoints[0].cpu().numpy().tolist()  # Convert to numpy array and then to list

        # Run SAM model with bounding boxes
        resultsam = modelsam(image_path, keypoints=keypoints_list)

        # Save the annotated image
        result = resultsam[0]
        output_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_a.png")
        result.save(output_path)
        print(f"{output_path} saved. \n")

In [ ]:
import os

# Define input and output directories
input_dir = "./marmoset_test/marmoset-dlc-2021-05-07/labeled-data/reachingvideo1"
output_dir = "./marmoset_test/marmoset-dlc-2021-05-07/labeled-data/output"

# Create output directory if it doesn't exist
if os.path.exists(output_dir) == False:
    os.makedirs(output_dir)

for filename in os.listdir(input_dir):
    if filename.lower().endswith(('.png')):
        # Path to the image
        image_path = os.path.join(input_dir, filename)

        # Run YOLO model
        model_output = modelyolo(image_path)
        boxes = model_output[0].boxes.xyxy

        # Check bounding boxes
        if len(boxes) == 0:
            print(f"No {filename}")
            continue

        # Convert bounding boxes to list format suitable for SAM
        bboxes_list = boxes[0].cpu().numpy().tolist()  # Convert to numpy array and then to list

        # Run SAM model with bounding boxes
        resultsam = modelsam(image_path, bboxes=bboxes_list)

        # Save the annotated image
        result = resultsam[0]
        output_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_a.png")
        result.save(output_path)
        print(f"{output_path} saved. \n")